In [ ]:
# import sqlite3
# import pandas as pd
# from services.connection_to_database import return_session,return_engine
# from services.schema_generation import  return_result_after_query,return_schema_short
# import sqlite3
# import pandas as pd
# import pandas as pd

# # Path to the SQLite database file (it will be created if it doesn't exist)
# sqlite_db_path = 'data/hiring_mod.sqlite'

# # Connect to the SQLite database (this will create the database if it doesn't exist)
# conn = sqlite3.connect(sqlite_db_path)
# cursor = conn.cursor()

# # Create a 'hiring' table with specified columns and set 'id' column as primary key
# create_table_query = '''
# CREATE TABLE IF NOT EXISTS hiring (
#     id INTEGER PRIMARY KEY,
#     title TEXT,
#     description TEXT,
#     name TEXT,
#     dateAddedFormatted DATE,
#     title_description TEXT,
#     skills_extracted TEXT,
#     categories TEXT,
#     sub_categories TEXT
# );
# '''


# cursor.execute(create_table_query)
# conn.commit()
# conn.close()

# db_path='sqlite:///data/hiring_mod.sqlite'
# session=return_session(db_path)
# engine=return_engine(db_path)
# schema=return_schema_short(engine)
# print(schema)

# df=pd.read_csv('data/hiring.csv')
# df.reset_index( inplace=True)
# print(df.columns)

# cols=df.columns
# for i,j in df.iterrows():
#     temp=[]
#     for k in cols:
#         temp.append(j[k])
#     #print(temp)
#     query = f"""
#     INSERT INTO hiring
#     (id, title, description, name, dateAddedFormatted, title_description, skills_extracted, categories, sub_categories)
#     VALUES ({temp[0]}, '{temp[1]}', '{temp[2]}', '{temp[3]}', '{temp[4]}', '{temp[5]}', '{temp[6].replace("'",'')}', '{temp[7]}', '{temp[8]}');
#     """
#     #print(query)
#     try:
#         print(temp[0])
#         sqlite_db_path = 'data/hiring_mod.sqlite'
#         conn = sqlite3.connect(sqlite_db_path)
#         cursor = conn.cursor()
#         cursor.execute(query)
#         conn.commit()
#         conn.close()
#     except Exception as e:
#         print("error",e)


In [ ]:

conn = sqlite3.connect("./data/hire.sqlite")
cursor = conn.cursor()
query="""
UPDATE hiring
SET dateAddedFormatted =  DATE(SUBSTR(dateAddedFormatted, 7, 4) || '-' || SUBSTR(dateAddedFormatted, 4, 2) || '-' || SUBSTR(dateAddedFormatted, 1, 2));
"""
cursor.execute(query)
conn.commit()
conn.close()

In [ ]:


Select 
strftime('%m',DATE(SUBSTR(dateAddedFormatted, 7, 4) || '-' || SUBSTR(dateAddedFormatted, 4, 2) || '-' || SUBSTR(dateAddedFormatted, 1, 2)))

AS formatted_date FROM hiring;
UPDATE hiring
SET dateAddedFormatted = strftime('%m', DATE(SUBSTR(dateAddedFormatted, 7, 4) || '-' || SUBSTR(dateAddedFormatted, 4, 2) || '-' || SUBSTR(dateAddedFormatted, 1, 2)));

In [ ]:
db_path="sqlite:///data/hire.sqlite"
session=return_session(db_path)
query="""

SELECT dateAddedFormatted,strftime('%m',dateAddedFormatted) as total FROM hiring ;


"""
t=return_result_after_query(session,query)
print(t)

In [ ]:
import sqlite3
import pandas as pd
from services.connection_to_database import return_session,return_engine
from services.schema_generation import  return_result_after_query,return_schema_short

from services.question_to_viz import question_to_viz,prompt_to_questions,return_schema_in_database
from services.connection_to_database import connectdatabasename_to_database,return_all_databases,connectdatabasename_to_database,return_engine,return_session
from services.schema_generation import  return_schema_short


databases=return_all_databases()
database_name='hiring'
# database_name="e_government"
db_path=connectdatabasename_to_database(database_name)

database_name,db_path


In [ ]:
#schema=return_schema_short_with_input(db_path,limit=2)
engine=return_engine(db_path)
schema=return_schema_short(engine)
print(schema)

In [ ]:
schema="""
'\nTable hiring, Columns= [id:INTEGER,title:TEXT,description:TEXT,name:TEXT,
dateAddedFormatted:DATE,title_description:TEXT,skills_extracted:TEXT,categories:TEXT,sub_categories:TEXT];'
"""

description_database="""
(id:unique primary key, title:job title for which recruitment needed, description:job description, name:name of company, dateAddedFormatted,
title_description:description about job title, skills_extracted:skill needed, categories:job under which it will lies, sub_categories)
"""

In [ ]:
example="""
{
"summary"="This database has information about ....",
"Roles"= ["HR Manager","Finance Manager","Sales Manager","Marketing Manager","IT Administrator","Customer Support Manager","Product Manager"]
}
"""
prompt=f"""

Please provide a brief summary of the database schema and suggest roles that users might assume while asking question for query 
a database which schema below in triple backticks.


Give a brief summary.
Database Schema:```{schema}```

please follow json format for giving output:
example:
{example}



"""
print(prompt)


In [ ]:
from services.connection_to_gpt import get_completion
import regex as re
model='gpt-3'
response,all_resp = get_completion(prompt,model=model)
print(response)

usd=83.25
if(model=='gpt-3'):
    print(f"total token:{all_resp['usage']['total_tokens']}")
    print(f"cost : {all_resp['usage']['total_tokens']*2*0.000001*usd} indian rupees")
elif (model=='gpt-4'):
    print(f"total token:{all_resp['usage']['total_tokens']}")
    print(f"cost : {all_resp['usage']['total_tokens']*6*0.00001*usd} indian rupees")
    


In [ ]:
from services.connection_to_gpt import get_completion
import regex as re
model='gpt-4'
response,all_resp = get_completion(prompt,model=model)
print(response)

usd=83.25
if(model=='gpt-3'):
    print(f"total token:{all_resp['usage']['total_tokens']}")
    print(f"cost : {all_resp['usage']['total_tokens']*2*0.000001*usd} indian rupees")
elif (model=='gpt-4'):
    print(f"total token:{all_resp['usage']['total_tokens']}")
    print(f"cost : {all_resp['usage']['total_tokens']*6*0.00001*usd} indian rupees")
    


In [ ]:
import json
summary,roles=json.loads(response)['summary'],json.loads(response)['Roles']
roles,summary

In [ ]:
role="HR Manager"

prompt= f"""
Your task is read and analyze the given schema and based on that create 10 different questions that should give excellent  
analytical insight to the user who is a {role} . It would be good, if you include questions that require SQL to use, Group by, aggregate 
function, min, max, probability etc. I want question to be more mathematical and analytical oriented.
Database Schema:```{schema}```

Give answer in Python list format: ["question1","question2","question3",...,"question10"]
 
"""
print(prompt)

In [ ]:
from services.connection_to_gpt import get_completion
import regex as re
import json
model='gpt-4'
response,all_resp = get_completion(prompt,model=model)
print(response)

usd=83.25
if(model=='gpt-3'):
    print(f"total token:{all_resp['usage']['total_tokens']}")
    print(f"cost : {all_resp['usage']['total_tokens']*2*0.000001*usd} indian rupees")
elif (model=='gpt-4'):
    print(f"total token:{all_resp['usage']['total_tokens']}")
    print(f"cost : {all_resp['usage']['total_tokens']*6*0.00001*usd} indian rupees")



questions=json.loads(response)
if len(questions)>9:
    print(questions)
else:
    questions=re.findall(r'\d+\.\s+(.+\?)',response)
    print(questions)

In [ ]:

question=questions[6]
question="What is the most common skill required in the job postings?"
print(question)

In [ ]:
from services.prompt_template import return_prompt_template

prompt=return_prompt_template(question,schema)
print(prompt)

In [ ]:



model='gpt-4'
response,all_resp = get_completion(prompt)
print(response)

usd=83.25
if(model=='gpt-3'):
    print(f"total token:{all_resp['usage']['total_tokens']}")
    print(f"cost : {all_resp['usage']['total_tokens']*2*0.000001*usd} indian rupees")
elif (model=='gpt-4'):
    print(f"total token:{all_resp['usage']['total_tokens']}")
    print(f"cost : {all_resp['usage']['total_tokens']*6*0.00001*usd} indian rupees")

In [ ]:


import regex as re
res=response

query=re.findall("'query':'(.*;)",res,re.DOTALL)[0]
print(query)
chart_type=re.findall("'chart_type':'(.*)',",res)[0]
print(chart_type)
code=re.findall("'chart_code':'(.*)'",res,re.DOTALL)[0]
print(code)

In [ ]:
from services.schema_generation import return_schema,return_result_after_query_in_df,return_result_after_query
t=return_result_after_query_in_df(session,query)
print(return_result_after_query(session,query))
print(t)

for i in t.columns:
    print(f'["{i}"]')
    code=code.replace(f'["{i}"]',str(list(t[i])))
    
code=code.replace("'",'"')
print(code)

In [ ]:
import json
import plotly.graph_objects as go

input_string = code

# Parse the input string into a dictionary
input_data = json.loads(input_string)

# Extract data and layout from the dictionary
data = input_data.get('data', [])
layout = input_data.get('layout', {})

# Create the Plotly figure
fig = go.Figure(data=data, layout=layout)

# Show the plot
fig.show()
